# 1192. Critical Connections in a Network

## Topic Alignment
- Finding bridges pinpoints single points of failure in large-scale services, an essential analysis for MLOps dependency graphs and data pipeline resilience.


## Metadata 摘要
- Source: https://leetcode.com/problems/critical-connections-in-a-network/
- Tags: Graph, DFS, Tarjan
- Difficulty: Hard
- Priority: High

## Problem Statement 原题描述
Given an undirected connected graph with n nodes and a list of edges, return all critical connections (bridges). Removing a bridge increases the number of connected components.

## Progressive Hints
- Hint 1: Run DFS and record discovery time `tin[u]` for each node.
- Hint 2: Maintain `low[u]`, the earliest discovery time reachable from `u` through tree edges plus at most one back edge.
- Hint 3: An edge `(u, v)` is critical if `low[v] > tin[u]`; it means `v` cannot reach `u` or its ancestors without that edge.


## Solution Overview
Apply Tarjan's bridge-finding algorithm. Perform DFS, assigning monotonically increasing timestamps. For each outgoing edge `(u, v)`, recurse on unvisited neighbors, update `low[u]` with `low[v]`, and flag the edge as a bridge if `low[v] > tin[u]`. When the neighbor is already visited and not the parent, update `low[u]` with `tin[v]` to account for back edges.


## Detailed Explanation
1. Build an adjacency list for the undirected graph.
2. Maintain arrays `tin` and `low` sized `n`, initialized to `-1` to indicate unvisited nodes, plus a global timer starting at 0.
3. For each unvisited node, call `dfs(u, parent)`:
   - Set `tin[u] = low[u] = timer` and increment `timer`.
   - Iterate neighbors `v`:
     - Skip `parent` to avoid immediately retracing the tree edge.
     - If `tin[v] == -1`, recurse `dfs(v, u)`, then update `low[u] = min(low[u], low[v])`. If `low[v] > tin[u]`, record `(u, v)` as a critical connection.
     - Otherwise, update `low[u] = min(low[u], tin[v])` for back edges.
4. Collect every edge flagged as critical and return the list.


## Complexity Trade-off Table
| Approach | Time | Space | Notes |
| --- | --- | --- | --- |
| Tarjan DFS | O(n + m) | O(n + m) | Linear bridge detection |
| Remove-edge + BFS per edge | O(m * (n + m)) | O(n + m) | Too slow for dense graphs |
| Union-Find with offline queries | O(n + m α(n)) | O(n) | Works for batch deletions but less direct here |


In [ ]:
from collections import defaultdict
from typing import List

class Solution:
    def criticalConnections(self, n: int, connections: List[List[int]]) -> List[List[int]]:
        graph = defaultdict(list)
        for u, v in connections:
            graph[u].append(v)
            graph[v].append(u)

        tin = [-1] * n
        low = [0] * n
        time = 0
        bridges: List[List[int]] = []

        def dfs(u: int, parent: int) -> None:
            nonlocal time
            tin[u] = low[u] = time
            time += 1
            for v in graph[u]:
                if v == parent:
                    continue
                if tin[v] == -1:
                    dfs(v, u)
                    low[u] = min(low[u], low[v])
                    if low[v] > tin[u]:
                        bridges.append([u, v])
                else:
                    low[u] = min(low[u], tin[v])

        dfs(0, -1)
        return bridges


In [ ]:
tests = [
    (4, [[0,1],[1,2],[2,0],[1,3]], [[1,3]]),
    (2, [[0,1]], [[0,1]])
]
solver = Solution()
for n, edges, expected in tests:
    result = solver.criticalConnections(n, edges)
    assert sorted(map(sorted, result)) == sorted(map(sorted, expected))
print('All tests passed.')


## Complexity Analysis
- Time: O(n + m) because each edge is explored at most twice during DFS.
- Space: O(n + m) for the adjacency list, recursion stack, and auxiliary arrays.


## Edge Cases & Pitfalls
- The graph is guaranteed connected, but the algorithm also works on disconnected graphs if you iterate over all nodes.
- Use recursion limits or convert to an explicit stack for very large graphs to avoid stack overflow.
- Remember to skip the immediate parent when processing neighbors to prevent false bridges.


## Follow-up Variants
- Return bridges grouped by connected component to plan targeted redundancy.
- Support online addition/removal of edges using dynamic bridge-finding data structures.
- Extend to directed graphs by computing strongly connected components and then identifying edges between SCCs.


## Takeaways
- Discovery time plus low-link values form the backbone of Tarjan's algorithms for bridges and articulation points.
- Bridges indicate single points of failure; removing them splits the network.
- Linear-time detection scales to massive production graphs where naive per-edge removal is infeasible.


## Similar Problems
| Problem ID | Problem Title | Technique |
| --- | --- | --- |
| LC 1135 | Connecting Cities With Minimum Cost | Minimum spanning tree to ensure connectivity |
| LC 310 | Minimum Height Trees | Identify critical centers via topological trimming |
| LC 323 | Number of Connected Components in an Undirected Graph | Union-Find / DFS for component counting |
